In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jites\\Desktop\\deployment\\image-classification-project\\research'

In [15]:
os.chdir('../')

In [16]:
%pwd

'c:\\Users\\jites\\Desktop\\deployment\\image-classification-project'

In [1]:
from dataclasses import dataclass
from pathlib import Path

                                ## this is the same thing what i have mensioned in config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])           


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config

In [4]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    print('1')
    data_ingestion_config = config.get_data_ingestion_config()
    print('2')
    data_ingestion = DataIngestion(config=data_ingestion_config)
    print('3')
    data_ingestion.download_file()
    print('4')
    data_ingestion.extract_zip_file()
    print('complete')
except Exception as e:
    raise e

[2023-07-24 19:36:24,710: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-24 19:36:24,712: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-24 19:36:24,714: INFO: common: created directory at: artifacts]
1
[2023-07-24 19:36:24,715: INFO: common: created directory at: artifacts/data_ingestion]
2
3
[2023-07-24 19:36:30,466: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 14995874
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "29ec243caeab52b8d2080f204e11fdbfc60bfd82eab22455bd16f8a72b0e9a86"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F7A8:58355:2835C:31BD2:64BE8561
Accept-Ranges: bytes
Date: Mon, 24 Jul 2023 14:06:26 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830041-CCU
X-Cac